In [1]:
import gym
import torch
import pandas as pd
from sb3_contrib import TRPO, RecurrentPPO, TQC
import numpy as np
import random

In [2]:
class ClipReward(gym.RewardWrapper):
    def __init__(self, env):
        super().__init__(env)
        self.best_y = 6
        self.score = 0
        self.last_y = 6

    def reward(self, reward):
        ram = env.unwrapped.ale.getRAM()
        reward = 0
        '''
        reward -= (1/ram[14]) * 40


        if 90 <= ram[106] <= 100:
            reward -= 1
        if ram[103] > self.score:
            self.score = ram[103]
            reward += 1000
        if ram[14] > self.best_y:
            self.best_y = ram[14]
            reward += 3
        if self.last_y > ram[14]:
            reward -= ram[14]'''
        if 140 <= ram[106] <= 141:
            reward += 10000
        reward += ram[14]

        if ram[14] >= 172:
            self.best_y = 6
        return reward

In [3]:
def get_action_sample():
    x = random.randint(0, 101)
    if x < 90:
        return 1
    if x < 97:
        return 2
    return 0

In [12]:
env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", render_mode="human", difficulty=1, mode=7))
obs = env.reset()

for _ in range(1000):
    action = get_action_sample()
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
3
3
3
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
1000
0
0
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
0
3
3
3
3
3
3
3
3
3
3
1000
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
0
3
3
3
3
3
0
3
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
0
0
0
0
0
0
0
0
0
0
3
3
3
3
3
3
1000
0
0
0
0
0
0
3
3
3
3
3
3
3
3
3
3
3
3
3
3
0
3
3
3
3
3
3
3
0
0
3
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0

In [4]:
ITERATIONS = 4000
STEPS_PER_ITERATION = 2048

env = ClipReward(gym.make("ALE/Freeway-v5", obs_type="ram", difficulty=1, mode=7))
obs = env.reset()

model = TRPO("MlpPolicy", env, gamma=0.95, verbose=1, tensorboard_log="tensorlog")
#model = TRPO.load("trpo_1.zip", env=env)
model.learn(total_timesteps=ITERATIONS*STEPS_PER_ITERATION, log_interval=10)
model.save("trpo_1.zip")

model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=7))
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to tensorlog\TRPO_16
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 2.05e+03 |
|    ep_rew_mean            | 4.96e+04 |
| time/                     |          |
|    fps                    | 371      |
|    iterations             | 10       |
|    time_elapsed           | 55       |
|    total_timesteps        | 20480    |
| train/                    |          |
|    explained_variance     | 0        |
|    is_line_search_success | 1        |
|    kl_divergence_loss     | 0.00299  |
|    learning_rate          | 0.001    |
|    n_updates              | 9        |
|    policy_objective       | 0.0152   |
|    value_loss             | 1.01e+05 |
----------------------------------------
----------------------------------------
| rollout/                  |          |
|    ep_len_mean            | 2.05e+03 |
|    ep_rew_mea

In [6]:
model = TRPO.load("trpo_1.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=7))
obs = env.reset()
for _ in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

8
12
16
20
6
6
6
6
6
6
6
6
6
6
6
6
6
6
10
14
18
22
26
30
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
6
6
6
6
6
6
6
6
6
6
6
6
6
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
66
70
74
78
82
86
90
94
98
102
106
110
114
118
122
126
6
6
6
6
6
6
6
6
6
6
6
6
6
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
66
70
74
78
82
86
90
94
98
102
106
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20
24
28
32
36
40
44
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
116
120
124
6
6
6
6
6
6
6
6
6
6
6
6
6
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
6
6
6
6
6
6
6
6
6
6
6
6
6
6
10
14
18
22
26
30
34
38
42
46
50
54
58
62
66
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20
24
28
32
36
40
44
48
52
56
60
64
68
72
76
80
84
88
92
96
100
104
108
112
6
6
6
6
6
6
6
6
6
6
6
6
6
6
10
14
18
22
26
30
34
38
42
46
50
54
58
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20
24
28
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20
24
28
6
6
6
6
6
6
6
6
6
6
6
6
6
6
8
12
16
20


In [ ]:
model = TRPO.load("trpo_3.zip")

env = ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0))
obs = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    #env.render()
    #time.sleep(0.00001)
    print(reward)
    if done:
        obs = env.reset()

In [24]:
import gym
from gym.wrappers import Monitor

env = Monitor(ClipReward(gym.make("ALE/Freeway-v5", render_mode="human", obs_type="ram", difficulty=1, mode=0)), './video', force=True)
state = env.reset()

model = TRPO.load("trpo_1.zip", env=env)

done = False
for i in range(1000):
    action, _states = model.predict(state, deterministic=True)
    state, reward, done, info = env.step(action)
env.close()

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
